<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FPreprocessing/Preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

# **Import**

In [ ]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math
import random

from sklearn.model_selection import train_test_split

## Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation
!ls -al

/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation
total 12581
drwx------ 2 root root     4096 Sep 14 08:50 'Case Report'
drwx------ 2 root root     4096 Aug 16 08:53  Dataset
drwx------ 2 root root     4096 Sep 25 16:33 '# Lagacy'
drwx------ 2 root root     4096 Aug 17 05:49  Log
-rw------- 1 root root  2079705 Oct  5 09:52 '[Model Tester] Down Sampling.ipynb'
-rw------- 1 root root    97579 Oct  6 19:41  Preprocessor.ipynb
drwx------ 2 root root     4096 Sep 14 06:04 'Raw Dataset'
-rw------- 1 root root 10684905 Oct  6 19:38 'Surgical-Wound UNet.ipynb'


# **Grobal variable**

In [ ]:
# Path (Absolute path in essential)
MODEL_PATH = "/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation"

RAW_DATA_PATH = "/Raw Dataset"

WOUND_TRAIN_PATH = "/Wound/train"
WOUND_TEST_PATH = "/Wound/test"
CVC_INPUT_PATH = "/CVC-clinicDB/Original"
CVC_GT_PATH = "/CVC-clinicDB/Ground Truth"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 224
NORM_INPUT_H_SIZE = 224

# **Function**

In [ ]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [ ]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

<br>
<br>
<br>
<br>
<br>
<br>

# **Preprocess Performace Compare Test**


---

<br>

***공통사항***
* 원본 이미지 크기 = 224 * 224
* epoch = 100
* batch = 64
> UNet batch size = 4<br>
> Wound dataset batch size = 8
* lr = 1e-3
* Train:Val = 7:3
* Used model = UNet
<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 한다.
* Convolution에서 특징점을 잘 추출하기 때문에 색상영역이나, 스무딩/샤프닝 같은 필터 처리는 되려 성능저하 요소가 될 수 있다. 

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.

<br>

***현재 결과***

---




<br>
<br>
<br>
<br>
<br>
<br>

## [***Control Group***](https://www.nature.com/articles/s41598-020-78799-w)
> [Train code](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/train.py)
> <br>
> [Data generator](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/utils/io/data.py#L10)
> <br>

### Performance

* Epoch: 200
* Batch Size: 64
* LR: 1e-3
* Optimizer: Adam
* loss: Dice loss

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 200 / 200

Best TEST || DICE_LOSS 0.1305

Best loss in Train (epoch 199, loss 0.0966)
Best loss in Val(Lastest save model) (epoch 194, loss 0.1231)

Train std =  0.2151
Val std =  0.2475
```

<br>
<br>
<br>
<br>
<br>

### Arrange dataset
> Not change anything about input image(scar)

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1
# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## ***Exprimental Group***

### Case 1
> 1:1 비율 변환

#### Performance

* Epoch: 200
* Batch Size: 64
* LR: 1e-3
* Optimizer: Adam
* loss: Dice loss

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 200 / 200

Best TEST || DICE_LOSS 0.1377

Best loss in Train (epoch 189, loss 0.0876)
Best loss in Val(Lastest save model) (epoch 197, loss 0.1311)

Train std =  0.1525
Val std =  0.2000
```

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# Image enhance model
#sr = cv2.dnn_superres.DnnSuperResImpl_create()
#sr.readModel(PROJECT_PATH + '/superres/models/EDSR_x3.pb')
#sr.setModel('edsr', 3)

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)
    
    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]
    
    # Compare width and height for resize to bigger length
    if width > height:
        height = width
    else:
        width = height
    
    # Resize
    image = cv2.resize(image,
                        (width, height),
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (width, height),
                          interpolation=cv2.INTER_CUBIC)
    
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1
    
# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>

### Case 2
> 중앙점 평행 이동

#### Performance

* Epoch: 200
* Batch Size: 64
* LR: 1e-3
* Optimizer: Adam
* loss: Dice loss

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 200 / 200

Best TEST || DICE_LOSS 0.1557

Best loss in Train (epoch 195, loss 0.0876)
Best loss in Val(Lastest save model) (epoch 196, loss 0.1462)

Train std =  0.1801
Val std =  0.2045
```

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])


    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>

### Case 3
> 비율 고정, 최대 크기 변환

#### Performance

* Epoch: 200
* Batch Size: 64
* LR: 1e-3
* Optimizer: Adam
* loss: Dice loss

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 200 / 200

Best TEST || DICE_LOSS 0.1744

Best loss in Train (epoch 200, loss 0.1075)
Best loss in Val(Lastest save model) (epoch 186, loss 0.1600)

Train std =  0.1008
Val std =  0.1694
```

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>

### Case 4
> 데이터들의 대각선 길이 평균값으로 대각선 크기 변환

#### Performance

* Epoch: 200
* Batch Size: 64
* LR: 1e-3
* Optimizer: Adam
* loss: Dice loss

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 200 / 200

Best TEST || DICE_LOSS 0.2339

Best loss in Train (epoch 185, loss 0.1036)
Best loss in Val(Lastest save model) (epoch 129, loss 0.2387)

Train std =  0.2362
Val std =  0.2043
```

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
diagonal_lens = []
diagonal_cnt = 0

# Counting diagonal length
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)
    
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])
    
    # Calc diagonal
    diagonal_len = round(np.sqrt((width**2) + (height**2)))
    if diagonal_len not in diagonal_lens:
        diagonal_cnt += 1
        
    diagonal_lens.append(diagonal_len)

# Print histogram
plt.hist(diagonal_lens, 
         bins=diagonal_cnt, 
         edgecolor='black')
plt.show()

D_LEN = max(diagonal_lens, key=diagonal_lens.count)
print("최대 빈도 대각선 길이 = ", D_LEN)
    
# Init
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)
    
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]
    
    # Calc diagonal
    diagonal_len = round(np.sqrt((width**2) + (height**2)))
    
    # Calc resize rate
    resize_scale = D_LEN / diagonal_len
    
    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1
    
# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1